In [3]:
# -*- coding: utf-8 -*-
"""
Combine charge-assignment summary with best matches from a databank,
supporting MULTIPLE (rt_window, mz_tol, mass_tol) triplets in one run.

- Reads:
    df1: assignments_summary (must have: neutral_mass, bin (or 'bin '), matched_mz_list)
    df2: databank_with_ids (must have: rt_aligned, precursor_mz, MASS, Accession, PFR)

- For each row in df1, for each m/z in matched_mz_list:
    find the single best df2 row where ALL hold:
        |rt_aligned - bin|    <= rt_window
        |precursor_mz - m/z|  <= mz_tol
        |MASS - neutral_mass| <= mass_tol
  Then format:
      best_match_* : "[<mz>: <Accession>, <MASS_from_df2>, <PFR>] ..."  (PFR optional)
      matched_pfr_*: "[<PFR_or_null_per_mz> ...]"  (aligned with matched_mz_list)
      mode_pfr_*   : most common non-null PFR across matches (per row)
      mode_pfr_count_* : frequency (count) of that PFR (non-null only)
      mode_accession_* : most common Accession across matches (per row), shown only if mode_pfr_count_* >= MIN_MODE_PFR_COUNT

- Outputs:
    One CSV with multiple columns per tolerance triplet:
      best_match_rt<RT>_mz<MZ>_mass<MASS>
      matched_pfr_rt<RT>_mz<MZ>_mass<MASS>
      mode_pfr_rt<RT>_mz<MZ>_mass<MASS>
      mode_pfr_count_rt<RT>_mz<MZ>_mass<MASS>
      mode_accession_rt<RT>_mz<MZ>_mass<MASS>

Edit the 3 PATHS and the PARAM_SETS below before running.
"""

from __future__ import annotations
import os
import ast
from typing import Optional, Dict, List, Tuple, Any
from collections import Counter

import numpy as np
import pandas as pd


# ----------------------------
# CONFIG: edit these paths
# ----------------------------
CHARGE_FILE_PATH = r"F:/idbenchmark/assignments_with_quant_sums.csv"
DATABANK_PATH    = r"F:/idbenchmark/databank_pfr_clean.csv"
OUTPUT_PATH      = r"F:/idbenchmark/assignments_with_quant_sums_pfr.csv"

# Provide one or more (rt_window, mz_tol, mass_tol) triplets here.
PARAM_SETS: List[Tuple[float, float, float]] = [
    (55.0, 2.0, 90.0),
    # (30.0, 1.0, 50.0),
]

# Keep "null" placeholders in matched_pfr_* so positions align with mz_list.
PFR_KEEP_PLACEHOLDERS: bool = True

# Minimum frequency required to report mode PFR and mode Accession.
MIN_MODE_PFR_COUNT: int = 1


# ----------------------------
# Helpers
# ----------------------------
def _num(s: pd.Series) -> pd.Series:
    """Coerce to numeric, invalid → NaN."""
    return pd.to_numeric(s, errors="coerce")


def _to_scalar(x: Any) -> Any:
    """Flatten 0-d arrays and coerce numeric strings to float when possible."""
    if isinstance(x, np.ndarray) and x.ndim == 0:
        x = x.item()
    if isinstance(x, str):
        try:
            return float(x)
        except Exception:
            return x
    return x


def _safe_parse_list(val) -> List[float]:
    """Convert a string-repr list into a Python list of floats safely."""
    if isinstance(val, str):
        try:
            parsed = ast.literal_eval(val)
            if isinstance(parsed, (list, tuple, np.ndarray)):
                return [float(x) for x in parsed]
            return []
        except Exception:
            return []
    if isinstance(val, (list, tuple, np.ndarray)):
        try:
            return [float(x) for x in val]
        except Exception:
            return []
    return []


def _ensure_columns(df: pd.DataFrame, required: List[str]) -> None:
    missing = [c for c in required if c not in df.columns]
    if missing:
        raise KeyError(f"Missing required column(s): {missing}")


def _fmt_suffix(v: float) -> str:
    """
    Make a tidy string for column suffixes (avoid many decimals).
    e.g., 10 -> '10', 2.0 -> '2', 1.5 -> '1p5'
    """
    if float(v).is_integer():
        return f"{int(v)}"
    # Replace '.' with 'p' to keep it column-name friendly
    return str(v).replace('.', 'p')


def _mode_or_none(items: List[Any]) -> Optional[Any]:
    """
    Return the most common value in `items` excluding None/NaN.
    If tie, Counter.most_common returns first encountered top count.
    """
    clean = []
    for v in items:
        if v is None:
            continue
        if isinstance(v, float) and np.isnan(v):
            continue
        clean.append(v)
    if not clean:
        return None
    return Counter(clean).most_common(1)[0][0]


def _mode_and_count(items: List[Any]) -> Tuple[Optional[Any], int]:
    """
    Return (mode_value, count) over non-null items.
    If no non-null items, returns (None, 0).
    """
    clean = []
    for v in items:
        if v is None:
            continue
        if isinstance(v, float) and np.isnan(v):
            continue
        clean.append(v)
    if not clean:
        return None, 0
    val, cnt = Counter(clean).most_common(1)[0]
    return val, int(cnt)


def _mode_and_count_with_cutoff(
    pfrs: List[Any], accs: List[Any], min_count: int
) -> Tuple[Optional[Any], int, Optional[Any]]:
    """
    Return (mode_pfr, count, mode_accession) applying a frequency cutoff on PFR.
    If the mode PFR count < min_count, return (None, 0, None).
    Otherwise return (mode_pfr, count, mode_accession).
    """
    pfr_val, pfr_cnt = _mode_and_count(pfrs)
    acc_val = _mode_or_none(accs)

    if pfr_cnt < min_count:
        return None, 0, None
    return pfr_val, pfr_cnt, acc_val


# ----------------------------
# Core search
# ----------------------------
def search_best(
    df2: pd.DataFrame,
    rt_query: float,
    mz_query: float,
    mass_query: float,
    rt_window: float,
    mz_tol: float,
    mass_tol: float
) -> Optional[Dict]:
    """
    Return the single best match (row as dict) if ALL three criteria match:
      |rt - rt_query| <= rt_window
      |mz - mz_query| <= mz_tol
      |mass - mass_query| <= mass_tol
    Otherwise returns None.
    """
    d_rt   = (df2["rt_aligned"] - float(rt_query)).abs()
    d_mz   = (df2["precursor_mz"] - float(mz_query)).abs()
    d_mass = (df2["MASS"] - float(mass_query)).abs()

    mask = (d_rt <= rt_window) & (d_mz <= mz_tol) & (d_mass <= mass_tol)
    if not mask.any():
        return None

    cand = df2.loc[mask].copy()
    cand["score"] = (
        d_rt.loc[cand.index] / max(rt_window, 1e-12) +
        d_mz.loc[cand.index] / max(mz_tol, 1e-12) +
        d_mass.loc[cand.index] / max(mass_tol, 1e-12)
    )
    best_row = cand.sort_values("score", kind="mergesort").iloc[0]
    return best_row.to_dict()


# ---------- per-row collectors (single pass across m/z list) ----------
def _collect_matches_for_row(
    row: pd.Series,
    df2: pd.DataFrame,
    rt_window: float,
    mz_tol: float,
    mass_tol: float
) -> Tuple[List[str], List[Optional[Any]], List[Optional[str]], List[Optional[float]]]:
    """
    For a df1 row, iterate over matched_mz_list and collect:
      - mz_tokens for best_match string (aligned, with placeholders)
      - pfr_list  (aligned, None for missing/NaN)
      - acc_list  (aligned, None for no match)
      - mass_list (aligned, MASS from df2 if matched, else None)

    Returns (best_match_tokens, pfr_list, acc_list, mass_list).
    """
    neutral_mass   = row.get("neutral_mass", np.nan)
    retention_time = row.get("bin", row.get("bin ", np.nan))
    mz_list        = _safe_parse_list(row.get("matched_mz_list", []))

    if pd.isna(neutral_mass) or pd.isna(retention_time) or not mz_list:
        return [], [], [], []

    tokens: List[str] = []
    pfrs:   List[Optional[Any]]   = []
    accs:   List[Optional[str]]   = []
    masses: List[Optional[float]] = []

    for mz_value in mz_list:
        res = search_best(
            df2,
            rt_query=float(retention_time),
            mz_query=float(mz_value),
            mass_query=float(neutral_mass),
            rt_window=rt_window,
            mz_tol=mz_tol,
            mass_tol=mass_tol,
        )
        if res is not None:
            uniprot_id = res.get("Accession", "NA")
            mass_match = _to_scalar(res.get("MASS", neutral_mass))
            pfr_val    = _to_scalar(res.get("PFR", None))
            if pfr_val is None or (isinstance(pfr_val, float) and np.isnan(pfr_val)):
                tokens.append(f"{mz_value}: {uniprot_id}, {mass_match}")
                pfrs.append(None)
            else:
                tokens.append(f"{mz_value}: {uniprot_id}, {mass_match}, {pfr_val}")
                pfrs.append(pfr_val)
            accs.append(uniprot_id)
            masses.append(mass_match)
        else:
            tokens.append(f"{mz_value}: NA")
            pfrs.append(None)
            accs.append(None)
            masses.append(None)

    return tokens, pfrs, accs, masses


def best_match_formatter_from_tokens(tokens: List[str]) -> Optional[str]:
    if not tokens:
        return None
    return "[" + ", ".join(tokens) + "]"


def matched_pfr_from_list(pfrs: List[Optional[Any]], keep_placeholders: bool) -> Optional[str]:
    if not pfrs:
        return None
    if keep_placeholders:
        return "[" + ", ".join("null" if v is None else str(v) for v in pfrs) + "]"
    else:
        pruned = [str(v) for v in pfrs if v is not None]
        return "[" + ", ".join(pruned) + "]" if pruned else None


# ----------------------------
# Main
# ----------------------------
def main():
    # Load CSVs
    if not os.path.exists(CHARGE_FILE_PATH):
        raise FileNotFoundError(f"Not found: {CHARGE_FILE_PATH}")
    if not os.path.exists(DATABANK_PATH):
        raise FileNotFoundError(f"Not found: {DATABANK_PATH}")

    df1 = pd.read_csv(CHARGE_FILE_PATH)
    df2 = pd.read_csv(DATABANK_PATH)

    # Normalize df1 column names to handle accidental trailing spaces, capitalization, etc.
    df1.columns = [c.strip() for c in df1.columns]

    # Ensure required columns in both tables (with tolerant check for 'bin' / 'bin ')
    need_df1 = ["neutral_mass", "matched_mz_list"]
    _ensure_columns(df1, need_df1)
    if "bin" not in df1.columns and "bin " not in df1.columns:
        raise KeyError("df1 must contain 'bin' (or 'bin ').")

    # Ensure essential df2 columns (PFR required for the new output)
    _ensure_columns(df2, ["rt_aligned", "precursor_mz", "MASS", "Accession", "PFR"])

    # If df1 had 'bin ' originally, create 'bin' as an alias to simplify downstream code
    if "bin" not in df1.columns and "bin " in df1.columns:
        df1["bin"] = df1["bin "]

    # Pre-coerce df2 numerics once (for speed)
    df2 = df2.copy()
    df2["rt_aligned"]   = _num(df2["rt_aligned"])
    df2["precursor_mz"] = _num(df2["precursor_mz"])
    df2["MASS"]         = _num(df2["MASS"])
    # PFR can be numeric or categorical; try to coerce but keep strings if not
    try:
        df2["PFR"] = pd.to_numeric(df2["PFR"], errors="ignore")
    except Exception:
        pass

    # Build output columns per tolerance triplet
    for (rt_w, mz_t, mass_t) in PARAM_SETS:
        suffix = f"rt{_fmt_suffix(rt_w)}_mz{_fmt_suffix(mz_t)}_mass{_fmt_suffix(mass_t)}"

        match_col   = f"best_match_{suffix}"
        pfr_col     = f"matched_pfr_{suffix}"
        mode_pfr    = f"mode_pfr_{suffix}"
        mode_pfr_n  = f"mode_pfr_count_{suffix}"
        mode_acc    = f"mode_accession_{suffix}"

        best_tokens_series: List[List[str]] = []
        pfr_list_series:    List[List[Optional[Any]]] = []
        acc_list_series:    List[List[Optional[str]]] = []

        # Compute per-row tokens and lists in one pass
        for _, row in df1.iterrows():
            tokens, pfrs, accs, _masses = _collect_matches_for_row(
                row, df2, rt_window=rt_w, mz_tol=mz_t, mass_tol=mass_t
            )
            best_tokens_series.append(tokens)
            pfr_list_series.append(pfrs)
            acc_list_series.append(accs)

        # Populate columns
        df1[match_col] = [best_match_formatter_from_tokens(toks) for toks in best_tokens_series]
        df1[pfr_col]   = [matched_pfr_from_list(pfrs, keep_placeholders=PFR_KEEP_PLACEHOLDERS)
                          for pfrs in pfr_list_series]

        # Most common PFR + its frequency (with cutoff), and most common Accession (masked if cutoff not met)
        mode_results = [
            _mode_and_count_with_cutoff(pfrs, accs, min_count=MIN_MODE_PFR_COUNT)
            for pfrs, accs in zip(pfr_list_series, acc_list_series)
        ]
        df1[mode_pfr]   = [mr[0] for mr in mode_results]
        df1[mode_pfr_n] = [mr[1] for mr in mode_results]
        df1[mode_acc]   = [mr[2] for mr in mode_results]

    # Save single CSV containing all columns
    out_dir = os.path.dirname(OUTPUT_PATH) or "."
    os.makedirs(out_dir, exist_ok=True)
    df1.to_csv(OUTPUT_PATH, index=False)
    print(
        f"Saved with {len(PARAM_SETS)} sets of columns "
        f"(best_match / matched_pfr / mode_pfr / mode_pfr_count / mode_accession) → {OUTPUT_PATH}"
    )


if __name__ == "__main__":
    main()


C:\Users\benja\AppData\Local\Temp\ipykernel_8552\4011789488.py:318: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df2["PFR"] = pd.to_numeric(df2["PFR"], errors="ignore")


Saved with 1 sets of columns (best_match / matched_pfr / mode_pfr / mode_pfr_count / mode_accession) → F:/idbenchmark/assignments_with_quant_sums_pfr.csv
